In [ ]:
import pandas as pd

capacities = pd.DataFrame(
    [["seattle", 350], ["san-diego", 600]], columns=["city", "capacity"]
).set_index("city")

capacities

In [ ]:
demands = pd.DataFrame(
    [["new-york", 325], ["chicago", 300], ["topeka", 275]], columns=["city", "demand"]
).set_index("city")

demands

In [ ]:
distances = pd.DataFrame(
    [
        ["seattle", "new-york", 2.5],
        ["seattle", "chicago", 1.7],
        ["seattle", "topeka", 1.8],
        ["san-diego", "new-york", 2.5],
        ["san-diego", "chicago", 1.8],
        ["san-diego", "topeka", 1.4],
    ],
    columns=["from", "to", "distance"],
).set_index(["from", "to"])

distances

In [ ]:
freight_cost = 90

In [ ]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [ ]:
m = Container()

In [ ]:
i = Set(container=m, name="i", description="plants")
i.setRecords(capacities.index)

In [ ]:
j = Set(container=m, name="j", description="markets", records=demands.index)

In [ ]:
i.records

In [ ]:
j.records

In [ ]:
a = Parameter(
    container=m,
    name="a",
    domain=i,
    description="supply of commodity at plant i (in cases)",
    records=capacities.reset_index(),
)
a.records

In [ ]:
b = Parameter(
    container=m,
    name="b",
    domain=j,
    description="demand for commodity at market j (in cases)",
    records=demands.reset_index(),
)
b.records

In [ ]:
c = Parameter(
    container=m,
    name="c",
    domain=[i, j],
    description="cost per unit of shipment between plant i and market j",
)

In [ ]:
cost = freight_cost * distances / 1000
cost

In [ ]:
c.setRecords(cost.reset_index())
c.records

In [ ]:
d = Parameter(
    container=m,
    name="d",
    domain=[i, j],
    description="distance between plant i and market j",
    records=distances.reset_index(),
)
d.records

In [ ]:
c[i, j] = freight_cost * d[i, j] / 1000
c.records

In [ ]:
x = Variable(
    container=m,
    name="x",
    domain=[i, j],
    type="Positive",
    description="amount of commodity to ship from plant i to market j",
)

In [ ]:
supply = Equation(
    container=m, name="supply", domain=i, description="observe supply limit at plant i"
)
demand = Equation(
    container=m, name="demand", domain=j, description="satisfy demand at market j"
)

In [ ]:
supply[i] = Sum(j, x[i, j]) <= a[i]

In [ ]:
demand[j] = Sum(i, x[i, j]) >= b[j]

In [ ]:
obj = Sum((i, j), c[i, j] * x[i, j])

In [ ]:
transport = Model(
    m,
    name="transport",
    equations=[supply, demand],
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

In [ ]:
transport_2 = Model(
    m,
    name="transport2",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

In [ ]:
import sys

transport.solve(output=sys.stdout)

In [ ]:
x.records.set_index(["i", "j"])

In [ ]:
transport.objective_value